In [148]:
import os
import sys
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)

In [149]:
workspace = "D:\\AALTO UNIVERSITY\\B.SC ECON - IMMIGRATION & PUBLIC TRANSPORTATION\\IMMIGRANT PATTERN & PUBLIC TRANSPORT ACCESSIBILITY\\DATA"

district_joined_path = os.path.join(workspace, "joined_district.csv")
population_helsinki_path = os.path.join(workspace, "Population", "ASHVAVL3.csv")
population_espoo_path = os.path.join(workspace, "Population", "ASEVAVL3.csv")
population_vantaa_path = os.path.join(workspace, "Population", "ASVVAVL3.csv")

In [150]:
# process population data for Helsinki, Espoo and Vantaa
population_helsinki_df = pd.read_csv(population_helsinki_path, sep = ",", encoding = "ISO-8859-1", skiprows = 2)
population_helsinki_df = population_helsinki_df[population_helsinki_df['Alue'].str.match(r'^\d{3} \d{3}.*')]
population_helsinki_df['municipal'] = population_helsinki_df['Alue'].str.extract(r'^(\d{3})').astype(int)
population_helsinki_df['district'] = population_helsinki_df['Alue'].str.extract(r'^\d{3} (\d{3})').astype(int)

population_espoo_df = pd.read_csv(population_espoo_path, sep = ",", encoding = "ISO-8859-1", skiprows = 2)
population_espoo_df = population_espoo_df[population_espoo_df['Alue'].str.match(r'^\d{3} \d{3}.*')]
population_espoo_df['municipal'] = population_espoo_df['Alue'].str.extract(r'^(\d{3})').astype(int)
population_espoo_df['district'] = population_espoo_df['Alue'].str.extract(r'^\d{3} (\d{3})').astype(int)

population_vantaa_df = pd.read_csv(population_vantaa_path, sep = ",", encoding = "ISO-8859-1", skiprows = 2)
population_vantaa_df = population_vantaa_df[population_vantaa_df['Alue'].str.match(r'^\d{3} \d{2}.*')]
population_vantaa_df['municipal'] = population_vantaa_df['Alue'].str.extract(r'^(\d{3})').astype(int)
population_vantaa_df['district'] = population_vantaa_df['Alue'].str.extract(r'^\d{3} (\d{2})').astype(int)

# helsinki edge cases
population_helsinki_edge_df = pd.read_csv(population_helsinki_path, sep = ",", encoding = "ISO-8859-1", skiprows = 2)
match_row_start = ['091 ' + str(i) + '0 ' for i in range(1,10)]
population_helsinki_edge_df = population_helsinki_edge_df[
    population_helsinki_edge_df['Alue'].apply(lambda x: any(x.startswith(start) for start in match_row_start))
]
population_helsinki_edge_df['municipal'] = population_helsinki_edge_df['Alue'].str.extract(r'^(\d{3})').astype(int)
population_helsinki_edge_df['district'] = population_helsinki_edge_df['Alue'].str.extract(r'^\d{3} (\d{2})').astype(int)


# merge population data
population_df = pd.concat([population_espoo_df, population_helsinki_df, population_vantaa_df, population_helsinki_edge_df], ignore_index=True)
population_df = population_df.drop(['Ikä'], axis=1)

print(population_df)

                                    Alue          Äidinkieli    2013    2014    2015    2016    2017    2018    2019    2020  municipal  district
0             049 111 Pohjois-Leppävaara  Kaikki äidinkielet    6326    6358    6516    6569    6576    6537    6693    6596         49       111
1             049 111 Pohjois-Leppävaara      Suomi ja saame    4838    4813    4881    4843    4831    4735    4791    4685         49       111
2             049 111 Pohjois-Leppävaara              Ruotsi     241     227     222     227     227     226     214     203         49       111
3             049 111 Pohjois-Leppävaara           Muu kieli    1247    1318    1413    1499    1518    1576    1688    1708         49       111
4               049 112 Etelä-Leppävaara  Kaikki äidinkielet    6680    6851    6884    7067    7214    7308    7321    7179         49       112
5               049 112 Etelä-Leppävaara      Suomi ja saame    5507    5527    5482    5562    5623    5624    5511    5338

In [151]:
# Create a mapping dictionary for renaming languages
rename_dict = {
    "Kaikki äidinkielet": "all",
    "Suomi ja saame": "finnish_sami",
    "Ruotsi": "swedish",
    "Muu kieli": "other"
}

population_df['Äidinkieli'] = population_df['Äidinkieli'].map(rename_dict)

# Melt the dataframe
population_df_melted = population_df.melt(id_vars=['municipal', 'district', 'Äidinkieli', 'Alue'], var_name='Year', value_name='Value')

# Convert non-numeric values in 'Value' column to NaN
population_df_melted['Value'] = pd.to_numeric(population_df_melted['Value'], errors='coerce')

# Using pivot_table to reshape the dataframe
population_df_pivot = population_df_melted.pivot_table(index=['municipal', 'district', 'Alue', 'Year'], columns='Äidinkieli', values='Value', aggfunc='first').reset_index()

population_df_pivot['Alue'] = population_df_pivot['Alue'].str[7:].str.strip()

# format year column
population_df_pivot['Year'] = population_df_pivot['Year'].astype(int)

print(population_df_pivot)

Äidinkieli  municipal  district                        Alue  Year       all  finnish_sami    other  swedish
0                  49       111          Pohjois-Leppävaara  2013    6326.0        4838.0   1247.0    241.0
1                  49       111          Pohjois-Leppävaara  2014    6358.0        4813.0   1318.0    227.0
2                  49       111          Pohjois-Leppävaara  2015    6516.0        4881.0   1413.0    222.0
3                  49       111          Pohjois-Leppävaara  2016    6569.0        4843.0   1499.0    227.0
4                  49       111          Pohjois-Leppävaara  2017    6576.0        4831.0   1518.0    227.0
5                  49       111          Pohjois-Leppävaara  2018    6537.0        4735.0   1576.0    226.0
6                  49       111          Pohjois-Leppävaara  2019    6693.0        4791.0   1688.0    214.0
7                  49       111          Pohjois-Leppävaara  2020    6596.0        4685.0   1708.0    203.0
8                  49       

In [152]:
# load district data
district_joined_df = pd.read_csv(district_joined_path, sep = ";", encoding = "utf-8")

# filter kauniainen (municipality id 235)
district_joined_df = district_joined_df[district_joined_df["municipal"] != 235]

print(district_joined_df)

     OID_  Join_Count  TARGET_FID  municipal  district                          name  pt_m_tt_2013_5874358  pt_m_t_2013_5874358  pt_m_d_2013_5874358  pt_m_tt_2013_5878088  pt_m_t_2013_5878088  pt_m_d_2013_5878088  pt_m_tt_2013_5911249  pt_m_t_2013_5911249  pt_m_d_2013_5911249  pt_m_tt_2013_5934892  pt_m_t_2013_5934892  pt_m_d_2013_5934892  pt_m_tt_2013_5934906  pt_m_t_2013_5934906  pt_m_d_2013_5934906  pt_m_tt_2013_5952975  pt_m_t_2013_5952975  pt_m_d_2013_5952975  pt_m_tt_2013_5958354  pt_m_t_2013_5958354  pt_m_d_2013_5958354  pt_m_tt_2013_5961860  pt_m_t_2013_5961860  pt_m_d_2013_5961860  pt_m_tt_2013_5961890  pt_m_t_2013_5961890  pt_m_d_2013_5961890  pt_m_tt_2013_5961894  pt_m_t_2013_5961894  pt_m_d_2013_5961894  pt_m_tt_2013_5968821  pt_m_t_2013_5968821  pt_m_d_2013_5968821  pt_m_tt_2013_5975374  pt_m_t_2013_5975374  pt_m_d_2013_5975374  pt_m_tt_2013_5978639  pt_m_t_2013_5978639  pt_m_d_2013_5978639  pt_m_tt_2013_5980222  pt_m_t_2013_5980222  pt_m_d_2013_5980222  pt_m_tt_2013_59802

In [153]:
# Dropping the unnecessary columns
drop_columns = ['OID_', 'Join_Count', 'TARGET_FID', 'Shape_Leng', 'Shape_Length', 'Shape_Area']
district_joined_df = district_joined_df.drop(columns=drop_columns, errors='ignore')

# Use the name of the district as the unique identifier
district_joined_df['unique_id'] = district_joined_df['municipal'].astype(str) + "_" + district_joined_df['district'].astype(str) + "_" + district_joined_df['name'].astype(str)

# Melt the dataframe
district_joined_df_melted = district_joined_df.melt(id_vars=['unique_id'], var_name='Attribute', value_name='Value')

# Splitting the 'Attribute' column
district_joined_df_melted[['Column-1', 'Column-2', 'Column-3', 'Year', 'ID']] = district_joined_df_melted['Attribute'].str.split('_', expand=True)
district_joined_df_melted['Column'] = district_joined_df_melted['Column-1'] + "_" + district_joined_df_melted['Column-2'] + "_" + district_joined_df_melted['Column-3']

# Drop unnecessary columns
district_joined_df_melted.drop(columns=['Attribute', 'Column-1', 'Column-2', 'Column-3'], inplace=True)

# Keep only rows with `Column` == `pt_m_t`
district_joined_df_melted = district_joined_df_melted[district_joined_df_melted['Column'] == 'pt_m_t']

# Pivot the dataframe
district_joined_df_pivot = district_joined_df_melted.pivot_table(index=['unique_id', 'Year'], columns=['ID'], values='Value', aggfunc='first').reset_index()

# split the unique_id column into municipal and district
district_joined_df_pivot[['municipal', 'district', 'name_district']] = district_joined_df_pivot['unique_id'].str.split('_', expand=True)

# Drop the unique_id column
district_joined_df_pivot.drop(columns=['unique_id'], inplace=True)

# Format year, municipal and district columns
district_joined_df_pivot['Year'] = district_joined_df_pivot['Year'].astype(int)
district_joined_df_pivot['municipal'] = district_joined_df_pivot['municipal'].astype(int)
district_joined_df_pivot['district'] = district_joined_df_pivot['district'].astype(int)

print(district_joined_df_pivot)

ID   Year 5874358 5878088 5911249 5934892 5934906 5952975 5958354 5961860 5961890 5961894 5968821 5975374 5978639 5980222 5980260  municipal  district                 name_district
0    2013      22      26      28       0      11      14      35      14      30      32      29      21      29      30      34         49       111            Pohjois-Leppävaara
1    2015      21      25      23       0      13      12      33      14      29      31      28      20      28      29      31         49       111            Pohjois-Leppävaara
2    2018      40      28      22       0      11      12      38      19      29      33      31      20      28      32      27         49       111            Pohjois-Leppävaara
3    2013      22      26      28       0      11      14      35      12      30      32      29      21      29      29      31         49       112              Etelä-Leppävaara
4    2015      21      25      23       0      12      12      33      13      29      31      

In [154]:
print(population_df_pivot[['municipal', 'district', 'Year']].dtypes)
print(district_joined_df_pivot[['municipal', 'district', 'Year']].dtypes)

Äidinkieli
municipal    int32
district     int32
Year         int32
dtype: object
ID
municipal    int32
district     int32
Year         int32
dtype: object


In [155]:
merged_df = pd.merge(population_df_pivot, district_joined_df_pivot, left_on=['municipal', 'district', 'Year'], right_on=['municipal', 'district', 'Year'], how='outer')

# Drop 'unique_id' column
# merged_df.drop('unique_id', axis=1, inplace=True)

# Rename 'Alue' column to 'District'
merged_df.rename(columns={'Alue': 'name_population'}, inplace=True)

print(merged_df)

# Save merged_df to csv
merged_df.to_csv('../Data/merged_df.csv', index=False, encoding='iso-8859-1')

      municipal  district             name_population  Year       all  finnish_sami    other  swedish 5874358 5878088 5911249 5934892 5934906 5952975 5958354 5961860 5961890 5961894 5968821 5975374 5978639 5980222 5980260                 name_district
0            49       111          Pohjois-Leppävaara  2013    6326.0        4838.0   1247.0    241.0      22      26      28       0      11      14      35      14      30      32      29      21      29      30      34            Pohjois-Leppävaara
1            49       111          Pohjois-Leppävaara  2014    6358.0        4813.0   1318.0    227.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN                           NaN
2            49       111          Pohjois-Leppävaara  2015    6516.0        4881.0   1413.0    222.0      21      25      23       0      13      12      33      14      29      31      28      20      28      29      31            Pohjois-Lep

In [156]:
filtered_merged_df = merged_df

# Check for non-NaN before checking the value
filtered_merged_df = filtered_merged_df[filtered_merged_df['finnish_sami'].apply(lambda x: x is not np.nan and x > 0)]
filtered_merged_df = filtered_merged_df[filtered_merged_df['name_population'].apply(lambda x: x != 'Muut')]
filtered_merged_df = filtered_merged_df[filtered_merged_df['name_population'].apply(lambda x: "peruspiiri" not in x.lower())]

# Group by 'municipal' and 'district', and filter
def filter_non_nan(group):
    return not group['5961890'].isna().all()

filtered_merged_df = filtered_merged_df.groupby(['municipal', 'district']).filter(filter_non_nan)

print(filtered_merged_df)

# Save to file
filtered_merged_df.to_csv('../Data/filtered_merged_df.csv', index=False, encoding='iso-8859-1')

      municipal  district          name_population  Year      all  finnish_sami   other  swedish 5874358 5878088 5911249 5934892 5934906 5952975 5958354 5961860 5961890 5961894 5968821 5975374 5978639 5980222 5980260                 name_district
0            49       111       Pohjois-Leppävaara  2013   6326.0        4838.0  1247.0    241.0      22      26      28       0      11      14      35      14      30      32      29      21      29      30      34            Pohjois-Leppävaara
1            49       111       Pohjois-Leppävaara  2014   6358.0        4813.0  1318.0    227.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN                           NaN
2            49       111       Pohjois-Leppävaara  2015   6516.0        4881.0  1413.0    222.0      21      25      23       0      13      12      33      14      29      31      28      20      28      29      31            Pohjois-Leppävaara
3           

In [157]:
extrapolated_df = filtered_merged_df.fillna(method='ffill')

extrapolated_df['non_official_lang_percentage'] = extrapolated_df['other'] / extrapolated_df['all']

# Create a list of columns to find the minimum
access_columns = ['5874358', '5878088', '5911249', '5934892', '5934906', '5952975', '5958354', '5961860', '5961890', '5961894', '5968821', '5975374', '5978639', '5980222', '5980260']

# Calculate the minimum across the columns for each row
extrapolated_df['time_to_fastest_cbd'] = extrapolated_df[access_columns].min(axis=1)

# Create 6 digits id for each suburb by concatenating the 3 digits municipal code (left with 0) and the 3 digits district code (right with 0)
extrapolated_df['id'] = extrapolated_df['municipal'].astype(str).str.zfill(3) + extrapolated_df['district'].astype(str).apply(lambda x: '-' + x if len(x) == 2 else x)
print(extrapolated_df)

# Save the dataframe to a csv file
extrapolated_df.to_csv('../Data/extrapolated.csv', index=False)

      municipal  district          name_population  Year      all  finnish_sami   other  swedish  5874358  5878088  5911249  5934892  5934906  5952975  5958354  5961860  5961890  5961894  5968821  5975374  5978639  5980222  5980260                 name_district  non_official_lang_percentage  time_to_fastest_cbd      id
0            49       111       Pohjois-Leppävaara  2013   6326.0        4838.0  1247.0    241.0       22       26       28        0       11       14       35       14       30       32       29       21       29       30       34            Pohjois-Leppävaara                      0.197123                    0  049111
1            49       111       Pohjois-Leppävaara  2014   6358.0        4813.0  1318.0    227.0       22       26       28        0       11       14       35       14       30       32       29       21       29       30       34            Pohjois-Leppävaara                      0.207298                    0  049111
2            49       111       Pohjo